In [1]:
import os
os.getcwd()
#Z:\Documents\Machine Learning\Python Machine Learning\Machine Learning\nlp_bundle\deep_learning_for_nlp

'/gpfs/global_fs01/sym_shared/YPProdSpark/user/s040-a001467a4c91fb-5b251a72b990/notebook/work'

In [2]:
import requests, zipfile, io
#https://github.com/meethariprasad/phd/raw/master/assignments/NLP/Translation/hin.zip
r = requests.get("http://www.manythings.org/anki/hin-eng.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [3]:
os.listdir()

['english-hindi-train.pkl',
 'english-german.pkl',
 'enghindi.txt',
 'english-hindi.pkl',
 'model.h5',
 'english-german-train.pkl',
 'hin.txt',
 'english-hindi-both.pkl',
 'english-german-both.pkl',
 'english-hindi-test.pkl',
 'english-german-test.pkl',
 '_about.txt']

In [4]:
file=open("hin.txt",'r',encoding='utf-8')
content=file.read()
file.close()

In [5]:
content[1:100]

'elp!\tबचाओ!\nJump.\tउछलो.\nJump.\tकूदो.\nJump.\tछलांग.\nHello!\tनमस्ते।\nHello!\tनमस्कार।\nCheers!\tवाह-वाह!\nChe'

In [6]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# tokenize on white space
			line = line.split()
			# remove punctuation from each token
			line = [re_punc.sub('', w) for w in line]
			# remove tokens with numbers in them
			#line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
filename = 'hin.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-german.pkl')
# spot check
for i in range(10):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-german.pkl
[Help] => [बचाओ]
[Jump] => [उछलो]
[Jump] => [कूदो]
[Jump] => [छलांग]
[Hello] => [नमस्ते।]
[Hello] => [नमस्कार।]
[Cheers] => [वाहवाह]
[Cheers] => [चियर्स]
[Got it] => [समझे कि नहीं]
[Im OK] => [मैं ठीक हूँ।]


In [65]:
from pickle import load
from pickle import dump
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-german.pkl')

# reduce dataset size
n_sentences = 2000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:1950], dataset[1950:]
# save
save_clean_data(dataset, 'english-german-both.pkl')
save_clean_data(train, 'english-german-train.pkl')
save_clean_data(test, 'english-german-test.pkl')

Saved: english-german-both.pkl
Saved: english-german-train.pkl
Saved: english-german-test.pkl


In [10]:
test.shape

(50, 2)

In [66]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# load datasets
#dataset = load_clean_sentences('english-german-both.pkl')
#train = load_clean_sentences('english-german-train.pkl')
#test = load_clean_sentences('english-german-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
#trainY=trainY.reshape((trainY.shape[0], trainY.shape[1], -1))
print("train",trainY.shape,trainY.shape[0],trainY.shape[1])
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)
#testY=testY.reshape((testY.shape[0], testY.shape[1], -1))
print("test",testY.shape,testY.shape[0],testY.shape[1])

English Vocabulary Size: 1691
English Max Length: 9
German Vocabulary Size: 2171
German Max Length: 12
train (1950, 9, 1691) 1950 9
test (50, 9, 1691) 50 9


In [67]:
import numpy
# prepare training data: Flip
trainX = numpy.flip(trainX,1)
# prepare validation data: Flip
testX = numpy.flip(testX,1)

In [81]:
# define NMT model
from keras import regularizers
from keras import optimizers
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(Bidirectional(LSTM(n_units,activation='relu',dropout=0.3)))
	model.add(RepeatVector(tar_timesteps))
	model.add(Bidirectional(LSTM(n_units, activation='relu',return_sequences=True,dropout=0.3)))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	# compile model
	optimizer = optimizers.RMSprop(lr=0.001)
	model.compile(optimizer=optimizer, loss='categorical_crossentropy')
	# summarize defined model
	model.summary()
	#plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [83]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 400)
# fit model
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, 
          batch_size=30, 
          validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)
modelbkp=model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 12, 400)           868400    
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 800)               2563200   
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 9, 800)            0         
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 9, 800)            3843200   
_________________________________________________________________
time_distributed_13 (TimeDis (None, 9, 1691)           1354491   
Total params: 8,629,291
Trainable params: 8,629,291
Non-trainable params: 0
_________________________________________________________________
Train on 1950 samples, validate on 50 samples
Epoch 1/30
Epoch 00000: val_loss improved from inf to 4.06479, saving model to model.h

In [84]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from numpy import argmax

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append(raw_target.split())
		predicted.append(translation.split())

# load model
#modelbkp=model
model = load_model('model.h5')

# test on some training sequences
print('train')
evaluate_model(model, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, testX, test)

train
src=[मेरे पापा का घर है।], target=[This is my fathers house], predicted=[its is is my]
src=[मैं मैनेजर से मिलना चाह्ती हूँ।], target=[I want to see the manager], predicted=[i i to to to]
src=[मैं आसपड़ोस में रहता हूँ।], target=[I live near here], predicted=[i am a]
src=[वह कौनसी चिड़िया है], target=[Whats that bird], predicted=[the is]
src=[मैं उसकी सुरक्षा के लिए परेशान हूँ।], target=[Im anxious about his safety], predicted=[i was a a]
src=[क्या तुम मुझे सिखा सकते हो], target=[Can you teach me], predicted=[can you you you]
src=[वह पिछले साल क्योटो गया था।], target=[He visited Kyoto last year], predicted=[he he the the]
src=[घूमफिरकर बात मत करो सीधेसीधे बोलो।], target=[Stop beating around the bush], predicted=[the the in]
src=[गेट को खुला छोड़ दिया गया था।], target=[The gate was left open], predicted=[the the the the]
src=[मुझे उल्टी आ रही है।], target=[I feel like throwing up], predicted=[i like like]
test
src=[वह सहीग़लत में अंतर करना नहीं जानता।], target=[He has no sense of ri

from pickle import load
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X


# evaluate the skill of the model
def evaluate_model(model, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append(raw_target.split())
		predicted.append(translation.split())

# load datasets
dataset = load_clean_sentences('english-german-both.pkl')
train = load_clean_sentences('english-german-train.pkl')
test = load_clean_sentences('english-german-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])